In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import datetime
from pathlib import Path
from tadpole.models import SimpleSVM
from tadpole.utils import test_train_split

tadpoleD1D2File = Path("/Users/jvdzwaan/data/TADPOLE/data/TADPOLE_D1_D2.csv")

df = pd.read_csv(tadpoleD1D2File)

# remove all rows without
df = df[df["Ventricles"].notnull()]
df = df[df["DXCHANGE"].notnull()]
df = df[df["ADAS13"].notnull()]

train_df, test_df, ground_truth_df = test_train_split(df, 0.1)

model = SimpleSVM()
model.train(train_df)

first_patient_id = test_df['RID'][0]
prediction_orig = model.predict(test_df[test_df['RID'] == first_patient_id], datetime.datetime.now())
truth_orig = ground_truth_df[ground_truth_df['RID'] == first_patient_id]

# evaluate (prediction, truth)

* CognitiveAssessmentDate = ExamDate (source: https://github.com/tadpole-share/TADPOLE-eval/blob/c8d4e241bc143b858d9b8237aab92417d3e871e2/evaluation/makeLeaderboardDataset.py#L159)
* ScanDate = ExamDate (source: https://github.com/tadpole-share/TADPOLE-eval/blob/c8d4e241bc143b858d9b8237aab92417d3e871e2/evaluation/makeLeaderboardDataset.py#L162)
* Diagnosis = DXCHANGE with mapping (source: https://github.com/tadpole-share/TADPOLE-eval/blob/c8d4e241bc143b858d9b8237aab92417d3e871e2/evaluation/makeLeaderboardDataset.py#L160 and line 169-170)
* RID = Patient id (what is it used for? Why must it be set?)

In [ ]:
from tadpole.transformations import get_diagnosis

# make copies so we don't have to train all the time
prediction = prediction_orig.copy()
truth = truth_orig.copy()

# The Forecast date has to be a string
p = pd.DataFrame(prediction)
p['Forecast Date'] = p['Forecast Date'].astype(str)
print(p.dtypes)

# Add expected columns
truth['CognitiveAssessmentDate'] = truth['EXAMDATE']
truth['ScanDate'] = truth['EXAMDATE']

truth['Diagnosis'] = get_diagnosis(truth)

In [ ]:
prediction

In [ ]:
for c in ground_truth_df.columns:
    print(c)

In [ ]:
def outcomes_equal(result1, result2):
    """Method that checks whether two evaluation outcomes are equal.
    """
    for i1, i2 in zip(result1, result2):
        if isinstance(i1, pd.Series):
            r = i1.equals(i2)
        else:
            r = i1 == i1 or np.isnan(i1) and np.isnan(i2)

        if not r:
            return False
    return True

In [ ]:
from evaluation import evalOneSubmissionExtended, simpleEvalOneSubmissionExtended

p = pd.DataFrame(prediction)
p['Forecast Date'] = p['Forecast Date'].astype(str)

res1 = evalOneSubmissionExtended.evalOneSub(truth.copy(), p.copy())
res2 = simpleEvalOneSubmissionExtended.evalOneSub(truth.copy(), p.copy())
                           
assert outcomes_equal(res1, res2)

In [ ]:
res1

In [ ]:
res2

In [ ]:
for i, (i1, i2) in enumerate(zip(res1, res2)):
    print(i, i1==i2, type(i1))

In [ ]:
import numpy as np

np.nan == np.nan

The evaluation code expects as input:

* `d4Df`: Pandas data frame containing the D4 dataset
  * Required fields in the DataFrame:
    * 
* `subDf`: Pandas data frame containing user forecasts for D2 subjects.
  * Required fields in the DataFrame:
    * Diagnosis
    * ADAS13
    * Ventricles_ICV
    * Forecast Date (string in the format `%Y-%m`)
    * RID: Patient ID
    * CN relative probability
    * MCI relative probability
    * AD relative probability
    * ADAS13 50% CI lower
    * ADAS13 50% CI upper
    * Ventricles_ICV 50% CI lower
    * Ventricles_ICV 50% CI upper

In [ ]:
import datetime

a = datetime.datetime.now()
b = a.strftime('%Y-%m')
type(b)

In [ ]:
datetime.datetime.strptime(b, '%Y-%m')

In [ ]:
def average_time_between_visits(df):
    df1 = df[['RID', 'EXAMDATE']].copy()
    df1 = df1.sort_values(by=['RID', 'EXAMDATE'])
    df1['EXAMDATE'] = df1['EXAMDATE'].map(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    # Get future value from each row's next row
    df1["Future_EXAMDATE"] = df1['EXAMDATE'].shift(-1)
    # Drop each last row per patient
    df1 = df1.drop(df1.groupby('RID').tail(1).index.values)
    return (df1['Future_EXAMDATE']-df1['EXAMDATE']).mean()


In [ ]:
average_time_between_visits(df)

In [ ]:
truth